In [1]:
import numpy as np
from PIL import Image
import argparse
from pathlib import Path

In [2]:
def apply_floyd_steinberg_dithering(gray_array):
    """Apply Floyd-Steinberg dithering to a grayscale 2D array (0–255)."""
    gray_array = gray_array.astype(np.float32)
    height, width = gray_array.shape

    for y in range(height):
        for x in range(width):
            old = gray_array[y, x]
            new = 255 if old > 127 else 0
            gray_array[y, x] = new
            error = old - new

            if x + 1 < width:
                gray_array[y, x + 1] += error * (7 / 16)
            if x - 1 >= 0 and y + 1 < height:
                gray_array[y + 1, x - 1] += error * (3 / 16)
            if y + 1 < height:
                gray_array[y + 1, x] += error * (5 / 16)
            if x + 1 < width and y + 1 < height:
                gray_array[y + 1, x + 1] += error * (1 / 16)

    return np.clip(gray_array, 0, 255).astype(np.uint8)

In [3]:
def dither_image_to_cmyk(image):
    """Convert image to CMYK and apply dithering to each channel."""
    cmyk_image = image.convert('CMYK')
    c, m, y, k = cmyk_image.split()

    c = apply_floyd_steinberg_dithering(np.array(c))
    m = apply_floyd_steinberg_dithering(np.array(m))
    y = apply_floyd_steinberg_dithering(np.array(y))
    k = apply_floyd_steinberg_dithering(np.array(k))

    return c, m, y, k

In [4]:
def save_cmyk_tiff(c, m, y, k, output_path):
    """Save CMYK channels as a CMYK TIFF image."""
    cmyk_array = np.stack([c, m, y, k], axis=-1)
    image = Image.fromarray(cmyk_array, mode='CMYK')
    image.save(output_path, format='TIFF')
    print(f"Saved: {output_path}")

In [5]:
def process_single_file(input_path, output_path):
    """Process a single image file and save as CMYK TIFF."""
    image = Image.open(input_path)
    c, m, y, k = dither_image_to_cmyk(image)
    save_cmyk_tiff(c, m, y, k, output_path)

In [6]:
def process_folder(input_folder, output_folder):
    """Process all supported images in a folder."""
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    supported = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}

    for file in input_folder.iterdir():
        if file.suffix.lower() in supported:
            output_file = output_folder / f"{file.stem}_cmyk.tiff"
            print(f"Processing: {file.name}")
            process_single_file(file, output_file)

In [7]:
def main():
    parser = argparse.ArgumentParser(description="Dither RGB image to CMYK using Floyd-Steinberg and save as TIFF.")
    parser.add_argument("input", help="Input image file or folder")
    parser.add_argument("output", help="Output TIFF path or folder (for batch)")

    args = parser.parse_args()
    input_path = Path(args.input)

    if input_path.is_file():
        process_single_file(input_path, Path(args.output))
    elif input_path.is_dir():
        process_folder(input_path, args.output)
    else:
        raise FileNotFoundError(f"Input not found: {args.input}")

In [8]:
process_single_file("lucy.png","lucy.tiff")

Saved: lucy.tiff
